# Step 1. Exploratory Data Analysis
## 1.1 Data loading and initial inspection

In [1]:
import pandas as pd

path1 = '/datasets/project_sql_result_01.csv'
path4 = '/datasets/project_sql_result_04.csv'

df_companies = pd.read_csv(path1)
df_neighborhoods = pd.read_csv(path4)

print("Companies and trips (first 5 records):")
print(df_companies.head(), "\n")

print("Neighborhoods and average trips (first 5 records):")
print(df_neighborhoods.head(), "\n")

print("df_companies types:")
print(df_companies.dtypes, "\n")

print("df_neighborhoods types:")
print(df_neighborhoods.dtypes, "\n")


Companies and trips (first 5 records):
                      company_name  trips_amount
0                        Flash Cab         19558
1        Taxi Affiliation Services         11422
2                Medallion Leasing         10367
3                       Yellow Cab          9888
4  Taxi Affiliation Service Yellow          9299 

Neighborhoods and average trips (first 5 records):
  dropoff_location_name  average_trips
0                  Loop   10727.466667
1           River North    9523.666667
2         Streeterville    6664.666667
3             West Loop    5163.666667
4                O'Hare    2546.900000 

df_companies types:
company_name    object
trips_amount     int64
dtype: object 

df_neighborhoods types:
dropoff_location_name     object
average_trips            float64
dtype: object 



## 1.2 Ensure correct data types

In [2]:
top10_neighborhoods = (
    df_neighborhoods
    .sort_values('average_trips', ascending=False)
    .head(10)
    .reset_index(drop=True)
)

print("Top 10 neighborhoods by average trips:")
print(top10_neighborhoods)


Top 10 neighborhoods by average trips:
  dropoff_location_name  average_trips
0                  Loop   10727.466667
1           River North    9523.666667
2         Streeterville    6664.666667
3             West Loop    5163.666667
4                O'Hare    2546.900000
5             Lake View    2420.966667
6            Grant Park    2068.533333
7         Museum Campus    1510.000000
8            Gold Coast    1364.233333
9    Sheffield & DePaul    1259.766667


## 1.3 Identification of top 10 neighborhoods

In [3]:
top10_neighborhoods = (
    df_neighborhoods
    .sort_values('average_trips', ascending=False)
    .head(10)
    .reset_index(drop=True)
)

print("Top 10 neighborhoods by average trips:")
print(top10_neighborhoods)


Top 10 neighborhoods by average trips:
  dropoff_location_name  average_trips
0                  Loop   10727.466667
1           River North    9523.666667
2         Streeterville    6664.666667
3             West Loop    5163.666667
4                O'Hare    2546.900000
5             Lake View    2420.966667
6            Grant Park    2068.533333
7         Museum Campus    1510.000000
8            Gold Coast    1364.233333
9    Sheffield & DePaul    1259.766667


## 1.4 Visualizations

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

top10_companies = (
    df_companies
    .sort_values('trips_amount', ascending=False)
    .head(10)
)

plt.figure(figsize=(8, 5))
sns.barplot(
    data=top10_companies,
    x='trips_amount',
    y='company_name',
    palette='Blues_r'
)
plt.title('Top 10: Trips per company (Nov 15–16, 2017)')
plt.xlabel('Number of trips')
plt.ylabel('Company')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
sns.barplot(
    data=top10_neighborhoods,
    x='average_trips',
    y='dropoff_location_name',
    palette='Oranges_r'
)
plt.title('Top 10: Neighborhoods by average arrivals (Nov 2017)')
plt.xlabel('Average trips')
plt.ylabel('Neighborhood')
plt.tight_layout()
plt.show()


## 1.5 Conclusions
The companies chart clearly shows which company mobilized the most passengers during the weekend of November 15–16. A large difference between first and second place may indicate market dominance or point promotion campaigns, in this case, Flash Cab.

In the top 10 neighborhoods, districts with the highest activity (financial center, tourist areas) stand out, suggesting that passenger flow in November is concentrated in high-density commercial and transport areas, with the Loop being the most prominent.

Both of these insights allow Zuber to adjust its fleet coverage: reinforce the number of cars for leading companies during peak demand and park vehicles near the most active neighborhoods to reduce wait times.

# Step 2. Hypothesis Testing

## 2.1 Data loading and group separation

In [5]:
import pandas as pd
from scipy import stats

df = pd.read_csv('/datasets/project_sql_result_07.csv', parse_dates=['start_ts'])

durations_bad  = df.loc[df.weather_conditions == 'Bad',  'duration_seconds']
durations_good = df.loc[df.weather_conditions == 'Good', 'duration_seconds']


## 2.2 Hypothesis statement
Null hypothesis (H₀): the average duration is the same on rainy and non-rainy Saturdays.

Alternative hypothesis (H₁): the average duration differs between rainy and non-rainy Saturdays.

This bilateral approach allows us to detect an increase or a decrease in duration due to bad weather.

## 2.3 Test criteria and significance level
Significance level: α = 0.05.

Selected test: Student's t-test for independent samples with unequal variances (Welch's t-test).

Justification: by comparing two means of different groups and not assuming equal variances, Welch better controls for Type I error.

## 2.4 Test implementation

In [6]:
stat_levene, p_levene = stats.levene(durations_bad, durations_good)
equal_var = p_levene > 0.05  # True if we don't reject H0 from Levene

t_stat, p_value = stats.ttest_ind(
    durations_bad,
    durations_good,
    equal_var=equal_var
)

print(f"Levene p-value = {p_levene:.4f} → equal_var = {equal_var}")
print(f"t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")

Levene p-value = 0.5332 → equal_var = True
t-statistic = 6.9462, p-value = 0.0000


## 2.5 Interpretation of results
* The Levene test yielded p-value = 0.5332, so we do not reject equality of variances and can use the standard Student's t-test (equal_var=True).

* The t-test resulted in t-statistic = 6.9462 with p-value ≈ 0.0000.

Since p-value < 0.05, we reject the null hypothesis of equal means. This means that, with a significance level α = 0.05, there is very strong statistical evidence that the average duration of Loop → O’Hare trips differs on rainy Saturdays compared to Saturdays with good weather.

Additionally, since the t-statistic is positive and significant, we conclude that trips on rainy Saturdays (Bad) are on average longer than on Saturdays with good weather (Good). This supports the idea that rain generates delays and longer travel times.